## Clique aqui para executar tudo! Não esqueça de trocar seu 'access_token' e mês que deseja consultar. Aqui temos o Cronograma de envio '1200479187862815'

In [ ]:
pip install asana

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 2.3 MB/s eta 0:00:00


In [ ]:
import asana
from asana.rest import ApiException
from pprint import pprint

In [ ]:
import requests
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Configure OAuth2 access token for authorization
configuration = asana.Configuration()
configuration.access_token = 'xxxxx' #troque pelo seu acesso
api_client = asana.ApiClient(configuration)

# Criando uma instância da API de Tarefas
tasks_api_instance = asana.TasksApi(api_client)

# ID do projeto (substitua pelo ID do projeto desejado)
project_id = 'xxxxxxx' # id cronograma de envio

In [ ]:
# Definindo os campos desejados para as tarefas, incluindo o campo personalizado "Tipo de Comunicação"
opts = {
    'opt_fields': ",".join([
        "gid",
        "created_at",
        "completed_at",
        "modified_at",
        "name",
        "assignee.name",
        "due_on",
        "tags.name",
        "notes",
        "projects.name",
        "custom_fields",
    ])
}
def get_task_data(task, parent_task_name=None):
    assignee_name = task.get('assignee', {}).get('name') if task.get('assignee') else None
    task_data = {
        'Nome da Tarefa': parent_task_name if parent_task_name else task.get('name'),
        'Responsável': assignee_name,
        'Concluída em': task.get('due_on'),
        'Notas': task.get('notes'),
        'Tags': [tag.get('name') for tag in task.get('tags')] if task.get('tags') else None
    }

    # Buscando o valor do campo personalizado "Tipo de Comunicação"
    tipo_comunicacao = None
    custom_fields = task.get('custom_fields', [])
    for field in custom_fields:
        if field.get('name') == 'Tipo de Comunicação':
            enum_value = field.get('enum_value')
            tipo_comunicacao = enum_value.get('name') if enum_value else None
            break  # Sair do loop assim que o campo for encontrado

    task_data['Tipo de Comunicação'] = tipo_comunicacao

    return task_data

# Obtendo as tarefas do projeto
try:
    project_tasks_response = tasks_api_instance.get_tasks_for_project(project_id, opts=opts)
    task_data = [get_task_data(task) for task in project_tasks_response]
except ApiException as e:
    print("Erro ao obter tarefas do projeto:", str(e))
    exit()

# Criando um DataFrame a partir dos dados coletados
df = pd.DataFrame(task_data)

df

Dados apresentados no relátorio

In [ ]:
# Remover linhas com valores NaN na coluna 'Concluída em'
df = df.dropna(subset=['Concluída em'])

# Filtrar as tarefas pelo mês e ano de conclusão
filtro_mes = df['Concluída em'].str.contains("2024-03-")

tarefas = df[filtro_mes]

colunas_desejadas = ["Nome da Tarefa", "Responsável", "Notas", "Concluída em", "Tags", "Tipo de Comunicação"]
tarefas = tarefas[colunas_desejadas]

tarefas.reset_index(drop=True, inplace=True)


# Salvar o DataFrame em um arquivo CSV
nome_arquivo_csv = 'comunicacao.csv'
tarefas.to_csv(nome_arquivo_csv, index=False)

tarefas

In [ ]:
# Contar a quantidade de tarefas com cada tag
quantidade_email = tarefas['Tipo de Comunicação'].apply(lambda x: "E-mail" in x if x else False).sum()
quantidade_push = tarefas['Tipo de Comunicação'].apply(lambda x: "Push" in x if x else False).sum()
quantidade_comunique = tarefas['Tipo de Comunicação'].apply(lambda x: "Comunique" in x if x else False).sum()
# Contar o número de tarefas em tarefas_relatorio_setembro
numero_de_tarefas= tarefas.shape[0]

# Mostrar o resultado
print(f"Número de tarefas no mês: {numero_de_tarefas}, sendo:")
print(f"{quantidade_email} E-mails")
print(f"{quantidade_push} Pushes")
print(f"{quantidade_comunique} Comuniques")



Número de tarefas no mês: 9, sendo:
5 E-mails
4 Pushes
0 Comuniques
